# 数据集准备

In [1]:
from PIL import Image
import numpy as np
import torch
import os
import torchvision.transforms as transforms
import random
import numpy.random as rng

In [2]:
img_size=30

In [3]:
labels=[]
images=[]
for i in range(460):
    m=i+1
    samples_num=os.listdir(os.path.join('D:/数据库/palmdata/iitd',str(m)))
    imgs=[]
    labs=[]
    for k in range(5):  #len(samples_num)
        j=k+1
        tpath=os.path.join(r'D:/数据库/palmdata/iitd',str(m),str(j)+'.jpeg')     #路径(/home/ouc/river/test)+图片名（img_m）
        fopen = Image.open(tpath)
        transform=transforms.Compose([transforms.Resize((img_size, img_size)),  # 将图片缩放到指定大小（h,w）或者保持长宽比并缩放最短的边到int大小
                                         transforms.Grayscale(),
                                         transforms.ToTensor()]) 
        data=np.array(transform(fopen))#data就是预处理后，可以送入模型进行训练的数据了
        imgs.append(data)
        labs.append(m)
    labels.append(labs)
    images.append(imgs)

labels=np.array(labels)
images=np.array(images)
print(images.shape) #(460,5)
print(labels.shape)   #(460,5,1, 28, 28)

# train_x=images[:2025]   #前360类数据作为训练集
# train_y=labels[:2025]
# val_x=images[2025:2600]  #后100类数据作为测试集
# val_y=labels[2025:2600]

(460, 5, 1, 30, 30)
(460, 5)


In [4]:
#训练集
x_train=images[:360]
y_train=labels[:360]
#测试集
x_test=images[360:]
y_test=labels[360:]

In [5]:
#生成一个元batch数据，包含8个任务，每个任务输入batch_size=64的三元组
base_batch_size=80
num_tasks=8
def make_cache(images,labels,name='train'):
    classes,counts = np.unique(labels, return_counts=True)
    data_cache = [] 
    for _ in range(10): #1个epoch包含10个元batch
        tasks=[]  #保存一个元batch的数据，长度是num_tasks
        for _ in range(num_tasks):  #生成8个任务
            task=[] #保存一个任务的数据，长度是base_batch_size
            for i in range(base_batch_size):
                selected_cls = rng.choice(classes)   #选择第一个样本的类   #1~360,361~460
                genuine_imposter_ratio=random.randint(0,1)
                if genuine_imposter_ratio: #正例
                    cls_2=selected_cls
                else:  #负例
                    while True:
                        cls_2=random.choice(classes)
                        if cls_2!=selected_cls:
                            break
                ex1, ex2 = rng.choice(5,replace=False,size=(2,))   #0~4
                if name=='train':
                    img1=images[selected_cls-1][ex1]
                    img2=images[cls_2-1][ex2]
                else:
                    img1=images[selected_cls-361][ex1]
                    img2=images[cls_2-361][ex2]
                flag=genuine_imposter_ratio #返回0/1标记
                task.append((img1,img2,flag))
            task=np.array(task)
            tasks.append(task)
        tasks=np.array(tasks)
        #print(tasks.shape)  #(8, 80, 3)
        data_cache.append(tasks)
    return data_cache    #(10, 8, 80, 3)

#元网络的数据集
datasets_cache = {"train": make_cache(x_train,y_train),
                  "test": make_cache(x_test,y_test,name='test')}

In [6]:
indexes = {"train": 0, "test": 0}  #记录当前取到第几batch
datasets={'train':x_train,'test':x_test}
datasets_y={'train':y_train,'test':y_test}

#迭代地取出一个个batch的数据集
def next(mode='train'):
    # update cache if indexes is larger than len(data_cache)
    if indexes[mode] >= len(datasets_cache[mode]):
        indexes[mode] = 0
        datasets_cache[mode] = make_cache(datasets[mode],datasets[mode],name=mode)

    next_batch = datasets_cache[mode][indexes[mode]]
    indexes[mode] += 1

    return next_batch

# 模型

In [7]:
#=======================================模型
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy, copy

#构建基学习器,学习一个任务,sp集（80，3），qu集（16，3）
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.vars = nn.ParameterList()  ## 包含了所有需要被优化的tensor参数w和b
        self.vars_bn = nn.ParameterList()  ##bn层参数

        # 第1个conv2d=================================
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 1, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        # 第1个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])   #标准化？

        # 第2个conv2d====================================
        # in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        # 第2个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])

        # 第3个conv2d====================================
        # in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        # 第3个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])

        # 第4个conv2d======================================
        # in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        # 第4个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])

        running_mean = nn.Parameter(torch.zeros(64), requires_grad=False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad=False)
        self.vars_bn.extend([running_mean, running_var])

        ##linear===========================================
        weight = nn.Parameter(torch.ones([5, 64]))
        bias = nn.Parameter(torch.zeros(5))
        self.vars.extend([weight, bias])
        
        ##联合两个输入的fc层
        weight = nn.Parameter(torch.ones([10, 64]))
        bias = nn.Parameter(torch.zeros(10))
        self.vars.extend([weight, bias])
        
        weight = nn.Parameter(torch.ones([64, 1]))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight, bias])
        

    #         self.conv = nn.Sequential(
    #             nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
    #             nn.BatchNorm2d(64),
    #             nn.ReLU(),
    #             nn.MaxPool2d(2),

    #             nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
    #             nn.BatchNorm2d(64),
    #             nn.ReLU(),
    #             nn.MaxPool2d(2),

    #             nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
    #             nn.BatchNorm2d(64),
    #             nn.ReLU(),
    #             nn.MaxPool2d(2),

    #             nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
    #             nn.BatchNorm2d(64),
    #             nn.ReLU(),
    #             nn.MaxPool2d(2),

    #             FlattenLayer(),
    #             nn.Linear(64,5)
    #         )

    def forward(self, x1,x2, params=None, bn_training=True):
        '''
        :bn_training: set False to not update
        :return:
        '''
        if params is None:
            params = self.vars
        x=[x1,x2]
        for i in range(2):
            weight, bias = params[0], params[1]  # 第1个CONV层
            x[i] = F.conv2d(x[i], weight, bias, stride=2, padding=2)
            weight, bias = params[2], params[3]  # 第1个BN层
            running_mean, running_var = self.vars_bn[0], self.vars_bn[1]
            x[i] = F.batch_norm(x[i], running_mean, running_var, weight=weight, bias=bias, training=bn_training)
            x[i] = F.max_pool2d(x[i], kernel_size=2)  # 第1个MAX_POOL层
            x[i] = F.relu(x[i], inplace=[True])  # 第1个relu
            
            weight, bias = params[4], params[5]  # 第2个CONV层
            x[i] = F.conv2d(x[i], weight, bias, stride=2, padding=2)
            weight, bias = params[6], params[7]  # 第2个BN层
            running_mean, running_var = self.vars_bn[2], self.vars_bn[3]
            x[i]= F.batch_norm(x[i], running_mean, running_var, weight=weight, bias=bias, training=bn_training)
            x[i]= F.max_pool2d(x[i], kernel_size=2)  # 第2个MAX_POOL层
            x[i]= F.relu(x[i], inplace=[True])  # 第2个relu
            
            weight, bias = params[8], params[9]  # 第3个CONV层
            x[i] = F.conv2d(x[i], weight, bias, stride=2, padding=2)
            weight, bias = params[10], params[11]  # 第3个BN层
            running_mean, running_var = self.vars_bn[4], self.vars_bn[5]
            x[i]= F.batch_norm(x[i], running_mean, running_var, weight=weight, bias=bias, training=bn_training)
            x[i]= F.max_pool2d(x[i], kernel_size=2)  # 第3个MAX_POOL层
            x[i]= F.relu(x[i], inplace=[True])  # 第3个relu
            
            weight, bias = params[12], params[13]  # 第4个CONV层
            x[i] = F.conv2d(x[i], weight, bias, stride=2, padding=2)
            x[i] = F.relu(x[i], inplace=[True])  # 第4个relu
            weight, bias = params[14], params[15]  # 第4个BN层
            running_mean, running_var = self.vars_bn[6], self.vars_bn[7]
            x[i]= F.batch_norm(x[i], running_mean, running_var, weight=weight, bias=bias, training=bn_training)
            x[i]= F.max_pool2d(x[i], kernel_size=2)  # 第4个MAX_POOL层
            
            x[i]= x[i].view(x[i].size(0), -1)  ## flatten
            weight, bias = params[16], params[17]  # linear
            x[i] = F.linear(x[i], weight, bias)
            
        x_two=torch.cat((x[0],x[1]),1)  
        weight, bias = params[18], params[19]   #联合两个输入的第一个fc层
        output=F.linear(x_two, weight, bias)
        output= F.relu(output, inplace=[True])
    
        weight, bias = params[20], params[21] 
        output=F.linear(output, weight, bias)
        output= F.sigmoid(output, inplace=[True])
        return output  #输出0/1

    def parameters(self):
        return self.vars

In [8]:
#构建元学习器，学习一个batch任务（8个任务为一个batch）
class MetaLearner(nn.Module):
    def __init__(self):
        super(MetaLearner, self).__init__()
        self.update_step = 5  ## 基学习器训练更新5次
        self.update_step_test = 5
        self.net = BaseNet().cuda()
        self.meta_lr = 2e-4  #元学习器的学习率
        self.base_lr = 4 * 1e-2  #基学习器的学习率
        self.meta_optim = torch.optim.Adam(self.net.parameters(), lr=self.meta_lr)  #元学习器的优化器

    def forward(datasets_cache):  #输入一个batch的数据=8个任务的数据，每执行一次这个函数，元学习器就更新一次  （8，80，3）
        #x_spt, y_spt, x_qry, y_qry=x_spt,torch.LongTensor(y_spt),torch.LongTensor(x_qry),torch.LongTensor(y_qry)
        # 初始化
        # 8         80              3
        task_num, base_batch_size, trip = datasets_cache.size()
    
        sp=datasets_cache[:][:64][:]
        qu=datasets_cache[:][64:][:]
        
        loss_list_qu = [0 for _ in range(self.update_step + 1)]   #[0,0,0,0,0,0]  #保存一个任务内，基学习器的初始损失加后面5次更新的损失
        correct_list = [0 for _ in range(self.update_step + 1)]   #[0,0,0,0,0,0]  #保存一个任务内，基学习器的初始准确数加后面5次更新的准确数
        #遍历一个batch内的每个任务，对每个任务训练更新基学习器5次
        for i in range(task_num):
            ## 一个任务上基学习器的第0步更新
            y_hat = self.net(sp[i][:][0], sp[i][:][1],params=None, bn_training=True)  # (ways * shots, ways) 用支持集，基学习器学习某一个任务

            loss = F.binary_cross_entropy(y_hat, sp[i][:][2]) #在支持集上，计算基学习器在某个任务的损失
            grad = torch.autograd.grad(loss, self.net.parameters()) #计算该任务的梯度
            tuples = zip(grad, self.net.parameters())  ## 将梯度和参数\theta一一对应起来
            # fast_weights这一步相当于求了一个\theta - \alpha*\nabla(L)
            fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], tuples))  #更新一次后的基学习器参数
            # 在query集上测试，计算准确率
            # 这一步使用更新前的数据
            with torch.no_grad():
                y_hat = self.net(qu[i][:][0], qu[i][:][1], self.net.parameters(), bn_training=True) #使用前一步的基学习器计算查询集上的损失和准确数
                loss_qry = F.binary_cross_entropy(y_hat, qu[i][:][2])
                loss_list_qry[0] += loss_qry
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
                correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                correct_list[0] += correct

            # 使用更新后的数据在query集上测试。
            with torch.no_grad():
                y_hat = self.net(qu[i][:][0], qu[i][:][1], fast_weights, bn_training=True)  #使用更新后的基学习器计算查询集上的损失和准确数
                loss_qry = F.binary_cross_entropy(y_hat, qu[i][:][2])
                loss_list_qry[1] += loss_qry
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
                correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                correct_list[1] += correct

            for k in range(1, self.update_step):   #再进行4次支持集上训练后更新基学习器的参数，并记录查询集上的验证损失和准确数
                y_hat = self.net(sp[i][:][0], sp[i][:][1], params=fast_weights, bn_training=True)
                loss = F.binary_cross_entropy(y_hat, sp[i][:][2])
                grad = torch.autograd.grad(loss, fast_weights)
                tuples = zip(grad, fast_weights)
                fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], tuples))

                y_hat = self.net(qu[i][:][0], qu[i][:][1], params=fast_weights, bn_training=True)
                loss_qry = F.binary_cross_entropy(y_hat, qu[i][:][2])
                loss_list_qry[k + 1] += loss_qry

                with torch.no_grad():
                    pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
                    correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                    correct_list[k + 1] += correct
        #         print('hello')

        #元学习器参数更新
        loss_qry = loss_list_qry[-1] / task_num  #计算最后一次更新基学习器（第五次更新）上查询集的平均每个任务的损失（表示这个batch的损失）===》作为外部元学习器的一次损失，更新一次元学习器的参数
        self.meta_optim.zero_grad()  # 梯度清零
        loss_qry.backward() #
        self.meta_optim.step()

        #计算当前这步的元学习器学到的基学习器对于一个任务1+（5次更新）的对应的查询集平均准确率和损失
        accs = np.array(correct_list) / (query_size * task_num)
        loss = np.array(loss_list_qry) / (task_num)
        return accs, loss  #返回每一步（基学习器在每个任务上更新共六步）的平均损失和准确数

    #对MAML训练得到的基学习器测试准确率
    def finetunning(self, task):  #输入一个任务的数据  (80,3)

        correct_list = [0 for _ in range(self.update_step_test + 1)]   #[0,0,0,0,0,0]
        sp=datasets_cache[:64][:]
        qu=datasets_cache[64:][:]
        
        new_net = deepcopy(self.net)
        y_hat = new_net(sp[:][0], sp[:][1])
        loss = F.binary_cross_entropy(y_hat, sp[:][2])
        grad = torch.autograd.grad(loss, new_net.parameters())
        fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], zip(grad, new_net.parameters())))

        # 在query集上测试，计算准确率
        # 这一步使用更新前的基学习器参数
        with torch.no_grad():
            y_hat = new_net(qu[:][0], qu[:][1], params=new_net.parameters(), bn_training=True)
            pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
            correct = torch.eq(pred_qry, y_qry).sum().item()
            correct_list[0] += correct

        # 使用更新后的基学习器参数。
        with torch.no_grad():
            y_hat = new_net(qu[:][0], qu[:][1], params=fast_weights, bn_training=True)
            pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
            correct = torch.eq(pred_qry, y_qry).sum().item()
            correct_list[1] += correct

        for k in range(1, self.update_step_test):  #五步更新基学习器（这里是剩下四步）
            y_hat = new_net(sp[:][0], sp[:][1], params=fast_weights, bn_training=True)
            loss = F.binary_cross_entropy(y_hat, sp[:][2])
            grad = torch.autograd.grad(loss, fast_weights)
            fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], zip(grad, fast_weights)))

            y_hat = new_net(qu[:][0], qu[:][1], fast_weights, bn_training=True)

            with torch.no_grad():
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
                correct = torch.eq(pred_qry, y_qry).sum().item()
                correct_list[k + 1] += correct

        del new_net
        accs = np.array(correct_list) / query_size  #返回一个任务上基学习器的平均准确率
        return accs

# 模型训练

In [9]:
import time
device = torch.device('cuda')

meta = MetaLearner().to(device)  #初始化元学习器

epochs =1000 # 60000
for step in range(epochs):
    start = time.time()
    datasets_cache = next('train')  #取出一个batch的数据（包含8个任务）
    #print(type(datasets_cache))   #<class 'numpy.ndarray'>
    #datasets_cache= torch.from_numpy(datasets_cache).to(device)
    datasets_cache[:][:][0]= torch.from_numpy(datasets_cache[:][:][0]).to(device)
    datasets_cache[:][:][1]= torch.from_numpy(datasets_cache[:][:][1]).to(device)
    datasets_cache[:][:][2]=torch.from_numpy(datasets_cache[:][:][2]).to(device)
#     print(type(datasets_cache[0][0]))
#     print(datasets_cache[0][0][0].shape)  #(1, 30, 30)
#     print(datasets_cache[0][0][1].shape)  #(1, 30, 30)
#     print(datasets_cache[0][0][2])   #0/1



    datasets_cache[:][:][:2]= torch.from_numpy(datasets_cache[:][:][:2]).to(device)
    datasets_cache[:][:][2]=torch.from_numpy(datasets_cache[:][:][2]).to(device)
                                 
    accs, loss = meta(datasets_cache)  #更新一次元学习器，返回这一次在八个任务上基学习器每一步更新得到的的损失和准确率
    end = time.time()

    if step % 20 == 0:  #每更新100次元学习器，输出一次当前的基学习器准确率和损失情况
        print("epoch:", step)
        print(accs)
        print(loss)

    if step % 30 == 0:  #每更新1000次元学习器，从测试集分别取出一个batch数据，从中依此取出一个任务的数据测试当前学到的基学习器
        accs = []
        for _ in range(1000 // task_num):
            # db_train.next('test')
            datasets_cache = next('test')
            datasets_cache[:][:][0]= torch.from_numpy(datasets_cache[:][:][0]).to(device)
            datasets_cache[:][:][1]= torch.from_numpy(datasets_cache[:][:][1]).to(device)
            datasets_cache[:][:][2]=torch.from_numpy(datasets_cache[:][:][2]).to(device)

            for task in datasets_cache:
                test_acc = meta.finetunning(task)
                accs.append(test_acc)
        print('在mean process之前：', np.array(accs).shape)
        accs = np.array(accs).mean(axis=0).astype(np.float16)
        print('测试集准确率:', accs)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [ ]:
#测试集准确率: [0.2072 0.8696 0.9287 0.939  0.9414 0.942 ]  800个epoch，tju1/tju2

In [ ]:
#测试集准确率: [0.2017 0.8677 0.9136 0.9175 0.9185 0.922 ]  1000个epoch tju1分割